### Imports

In [1]:
%%capture
!pip install transformers==4.20.0

In [2]:
%%capture
!pip install datasets

In [3]:
from tqdm import tqdm
import numpy as np
import pandas as pd
import os
import json
import spacy


#preparing
import datasets
from datasets import Dataset

#modelling

from transformers import TrainingArguments, Trainer
from transformers import BloomConfig, BloomModel
from transformers import AutoTokenizer

#evaluation 
import torch
from transformers import EvalPrediction
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction

### Constants

In [4]:
# connect with google drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
# change cwd
%cd drive/MyDrive/Work/Frontline/data

/content/drive/.shortcut-targets-by-id/1WfnZsqpG1r110J63sMbfS5TpsDOkveiV/data


### Data

In [6]:
dfs=[]
for doc in os.listdir("annotated"):
  if doc.startswith("annotations"):
    #read json data
    json_data=json.load(open("annotated/"+doc, encoding="utf-8"))
    #convert to dataframe
    data=pd.DataFrame(json_data["documents"])
    #for now: filter out paragraphs that have not been annotated 
    data=data[data["annotations"].apply(len)>0]    
    dfs.append(data)


In [7]:
# merge jsons
data=pd.concat(dfs)
data=data.reset_index(drop=True)

In [8]:
data["tags"]=[""]*data.shape[0]

In [9]:
# extract document labels
# ASSUMPTION: all annotations are document labels, ie. do not have start and end as those are ignored
 
for idx, entry in data.iterrows():
  # in case there are more than 1 label
  for label in range(len(entry["annotations"])):
    data.loc[idx, "tags"]+=entry["annotations"][label]["concept"]["preferred_label"]["name"]+"*"

Change old label names to new label names

In [10]:
data.tags=data.tags.str.replace("NA","Domestic Violence")
data.tags=data.tags.str.replace("Victim blaming","Statement of responsibility")

In [11]:
df=data[["tags", "text"]]

In [12]:
# converting the annotation column with one hot encoding
df=pd.merge(df,df["tags"].str.get_dummies(sep="*"), left_index=True, right_index=True)
# dropping the tags column
df=df.drop(labels="tags",axis=1)

In [13]:
# create a list of all labels
labels=df.columns.to_list()[1:]

### Split data into train and test

In [14]:
#split train, dev , test sets
df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42),
                            [int(.8 * len(df)), int(.9 * len(df))])

#### Convert DataFrame to  Dataset

In [15]:
ds=datasets.DatasetDict({"train":Dataset.from_dict(df_train),"val":Dataset.from_dict(df_val),"test":Dataset.from_dict(df_test)})

### Tokenizing & Encoding

In [16]:
tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-560m")

Downloading:   0%|          | 0.00/222 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.8M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

In [17]:
def preprocess_data(df):
  text = df["text"]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)
  # add labels
  
  labels_batch = {k: df[k] for k in df.keys() if k in labels}
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]
  encoding["labels"] = labels_matrix.tolist()
  return encoding

In [18]:
ds_encoded=ds.map(preprocess_data, batched=True, remove_columns=ds['train'].column_names)

Map:   0%|          | 0/433 [00:00<?, ? examples/s]

Map:   0%|          | 0/54 [00:00<?, ? examples/s]

Map:   0%|          | 0/55 [00:00<?, ? examples/s]

In [19]:
ds_encoded.set_format("torch")


### Modelling

In [20]:
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}

In [21]:
from transformers import AutoModelForSequenceClassification


In [22]:
model = AutoModelForSequenceClassification.from_pretrained("bigscience/bloom-560m",
                                   problem_type="multi_label_classification", 
                                   num_labels=len(labels),
                                   id2label=id2label,
                                   label2id=label2id)

Downloading:   0%|          | 0.00/693 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

Some weights of BloomForSequenceClassification were not initialized from the model checkpoint at bigscience/bloom-560m and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Evaluate

In [23]:
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
  
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

In [24]:
ds_encoded["train"]=ds_encoded["train"].rename_column("labels","label")
ds_encoded["test"]=ds_encoded["test"].rename_column("labels","label")
ds_encoded["val"]=ds_encoded["val"].rename_column("labels","label")


In [25]:
args = TrainingArguments(
    f"bloom-finetuned-tagging-german",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=6,
    per_device_eval_batch_size=6,
    num_train_epochs=10,
    weight_decay=0.05,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    #push_to_hub=True,
)

In [26]:
trainer = Trainer(
    model,
    args,
    train_dataset=ds_encoded["train"],
    eval_dataset=ds_encoded["val"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [27]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 433
  Num Epochs = 10
  Instantaneous batch size per device = 6
  Total train batch size (w. parallel, distributed & accumulation) = 6
  Gradient Accumulation steps = 1
  Total optimization steps = 730


Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,No log,0.875681,0.870370,0.913580,0.870370
2,No log,0.542268,0.870370,0.913580,0.870370
3,No log,0.738807,0.672727,0.783951,0.666667
4,No log,0.390434,0.834951,0.879630,0.796296
5,No log,0.661420,0.870370,0.913580,0.870370
6,No log,0.908925,0.870370,0.913580,0.870370
7,0.267400,1.030618,0.862385,0.910494,0.851852
8,0.267400,1.105094,0.851852,0.901235,0.851852
9,0.267400,1.166316,0.859813,0.904321,0.851852
10,0.267400,1.191348,0.870370,0.913580,0.870370


***** Running Evaluation *****
  Num examples = 54
  Batch size = 6
Saving model checkpoint to bloom-finetuned-tagging-german/checkpoint-73
Configuration saved in bloom-finetuned-tagging-german/checkpoint-73/config.json
Model weights saved in bloom-finetuned-tagging-german/checkpoint-73/pytorch_model.bin
tokenizer config file saved in bloom-finetuned-tagging-german/checkpoint-73/tokenizer_config.json
Special tokens file saved in bloom-finetuned-tagging-german/checkpoint-73/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 54
  Batch size = 6
Saving model checkpoint to bloom-finetuned-tagging-german/checkpoint-146
Configuration saved in bloom-finetuned-tagging-german/checkpoint-146/config.json
Model weights saved in bloom-finetuned-tagging-german/checkpoint-146/pytorch_model.bin
tokenizer config file saved in bloom-finetuned-tagging-german/checkpoint-146/tokenizer_config.json
Special tokens file saved in bloom-finetuned-tagging-german/checkpoint-146/special_tokens_

TrainOutput(global_step=730, training_loss=0.20195418841218296, metrics={'train_runtime': 2149.8648, 'train_samples_per_second': 2.014, 'train_steps_per_second': 0.34, 'total_flos': 1005338933329920.0, 'train_loss': 0.20195418841218296, 'epoch': 10.0})

## Predicting

In [28]:
sigmoid = torch.nn.Sigmoid()

In [29]:
test_pred=pd.DataFrame(ds["test"])

In [30]:
def predict_text(text):
  encoding = tokenizer(text, return_tensors="pt")
  encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}
  outputs = trainer.model(**encoding)
  logits = outputs.logits
  probs = sigmoid(logits.squeeze().cpu())
  predictions = np.zeros(probs.shape)
  predictions[np.where(probs >= 0.5)] = 1
  # turn predicted id's into actual label names
  predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
  return predicted_labels
  

In [31]:
pred=[]
for t in ds["test"]["text"]:
  pred.append(predict_text(t))
test_pred["prediction"]=pred

In [38]:
test_pred.head()

,text,Domestic Violence,Graphic,Sensationalist,Statement of responsibility,prediction
0,Beuel. Die Polizei ist am Samstagmorgen einer...,1,0,0,0,[Domestic Violence]
1,Westerwald. Seit 25 Jahren existiert das Fraue...,1,0,0,0,[Domestic Violence]
2,Die Bundesregierung nimmt das Thema ernst. Gef...,1,0,0,0,[Domestic Violence]
3,Für einige der Fälle liegen dem SÜDKURIER Doku...,1,0,0,0,[Domestic Violence]
4,Hamm (dpa/lnw) - Ein 33-Jähriger hat einer Fra...,1,0,0,0,[Domestic Violence]
